# Training CNN model

In [1]:
import os

import keras
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score

from utils.KerasModels import buildCharCNNModel
from utils.KerasModels import generateExpData

keras.utils.set_random_seed(812)
MODELS_DIR = os.path.join('..', 'models', 'charCNN')
EPOCHS = 200

In [2]:
def prepare_data(data_directory):
    train = pd.read_csv(os.path.join(data_directory, 'train.csv'))
    validation = pd.read_csv(os.path.join(data_directory, 'validation.csv'))
    test = pd.read_csv(os.path.join(data_directory, 'test.csv'))

    dfTrainDataset = train[["screen_name", "text", "account.type"]]
    dfValDataset = validation[["screen_name", "text", "account.type"]]
    dfTestDataset = test[["screen_name", "text", "account.type"]]

    tokenizer = None
    train_features, tokenizer = generateExpData(dfTrainDataset, tokenizer=tokenizer)
    val_features, tokenizer = generateExpData(dfValDataset, tokenizer=tokenizer)
    test_features, tokenizer = generateExpData(dfTestDataset, tokenizer=tokenizer)

    dictLabels = {"human": 0, "bot": 1}
    y_train = dfTrainDataset["account.type"].apply(lambda x: dictLabels[x])
    y_val = dfValDataset["account.type"].apply(lambda x: dictLabels[x])
    y_test = dfTestDataset["account.type"].apply(lambda x: dictLabels[x])

    train_labels = y_train.tolist()
    val_labels = y_val.tolist()
    test_labels = y_test.tolist()

    vocab_size = len(tokenizer.word_index)

    return train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size


def proba_to_pred(y_proba):
    y_pred_char_cnn = (y_proba > 0.5).astype(int)
    return y_pred_char_cnn


def calculate_metrics(y_true, y_pred):
    results = {
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred)
    }
    return results

In [3]:
DATA_PATH = os.path.join('..', 'data', 'preprocessed_url_simple')
train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size = prepare_data(DATA_PATH)
os.makedirs(MODELS_DIR, exist_ok=True)
model = buildCharCNNModel(vocab_size, embSize=32, inputSize=320)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 320, 32)      3360        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 318, 128)     12416       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 317, 128)     16512       ['embedding[0][0]']              
                                                                                              

In [4]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "simple"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
model.fit(np.array(train_features), np.array(train_labels), batch_size=256, epochs=EPOCHS,
          validation_data=(np.array(val_features), np.array(val_labels)),
          callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.5592 - accuracy: 0.7407

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 5s 22ms/step - loss: 0.5592 - accuracy: 0.7407 - val_loss: 0.4785 - val_accuracy: 0.7641
Epoch 2/200
79/81 [============================>.] - ETA: 0s - loss: 0.4348 - accuracy: 0.7928

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 1s 18ms/step - loss: 0.2696 - accuracy: 0.8834 - val_loss: 0.3430 - val_accuracy: 0.8467
Epoch 21/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2669 - accuracy: 0.8835 - val_loss: 0.3447 - val_accuracy: 0.8449
Epoch 22/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2576 - accuracy: 0.8890 - val_loss: 0.3520 - val_accuracy: 0.8228
Epoch 23/200
81/81 [==============================] - ETA: 0s - loss: 0.2557 - accuracy: 0.8885

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 1s 18ms/step - loss: 0.2557 - accuracy: 0.8885 - val_loss: 0.3415 - val_accuracy: 0.8371
Epoch 24/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2440 - accuracy: 0.8968 - val_loss: 0.3484 - val_accuracy: 0.8423
Epoch 25/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2412 - accuracy: 0.8976 - val_loss: 0.3501 - val_accuracy: 0.8427
Epoch 26/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2356 - accuracy: 0.9019 - val_loss: 0.3470 - val_accuracy: 0.8410
Epoch 27/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2311 - accuracy: 0.9040 - val_loss: 0.3507 - val_accuracy: 0.8384
Epoch 28/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2268 - accuracy: 0.9054 - val_loss: 0.3540 - val_accuracy: 0.8354
Epoch 29/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2215 - accuracy: 0.9084 - val_loss: 0.3598 - val_accuracy: 0.8371
Epoch 30/200

In [5]:
probas = model.predict(np.array(test_features))
y_pred = proba_to_pred(probas)
results = calculate_metrics(test_labels, y_pred)
results

80/80 [==============================] - 0s 1ms/step


{'balanced_accuracy': 0.8381302572378717,
 'f1_score': 0.843181818181818,
 'precision': 0.8183823529411764,
 'recall': 0.86953125}

# Lemmatized data 

In [6]:
LEMMATIZED_DATA_PATH = os.path.join('..', 'data', 'lemmatized')
lem_train_features, lem_val_features, lem_test_features, lem_train_labels, lem_val_labels, lem_test_labels, lem_vocab_size = prepare_data(
    LEMMATIZED_DATA_PATH)

modelLEM = buildCharCNNModel(lem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [7]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "lemmatized"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelLEM.fit(np.array(lem_train_features), np.array(lem_train_labels), batch_size=256, epochs=EPOCHS,
             validation_data=(np.array(lem_val_features), np.array(lem_val_labels)),
             callbacks=[early_stopping, save_best])

Epoch 1/200
80/81 [============================>.] - ETA: 0s - loss: 0.5575 - accuracy: 0.7444

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.5564 - accuracy: 0.7447 - val_loss: 0.4836 - val_accuracy: 0.7619
Epoch 2/200
78/81 [===========================>..] - ETA: 0s - loss: 0.4401 - accuracy: 0.7912

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 1s 18ms/step - loss: 0.2811 - accuracy: 0.8736 - val_loss: 0.3432 - val_accuracy: 0.8440
Epoch 19/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2736 - accuracy: 0.8784 - val_loss: 0.3446 - val_accuracy: 0.8423
Epoch 20/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2679 - accuracy: 0.8834 - val_loss: 0.3436 - val_accuracy: 0.8480
Epoch 21/200
78/81 [===========================>..] - ETA: 0s - loss: 0.2647 - accuracy: 0.8850

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 2s 19ms/step - loss: 0.2654 - accuracy: 0.8844 - val_loss: 0.3403 - val_accuracy: 0.8449
Epoch 22/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2557 - accuracy: 0.8886 - val_loss: 0.3504 - val_accuracy: 0.8262
Epoch 23/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2539 - accuracy: 0.8927 - val_loss: 0.3404 - val_accuracy: 0.8388
Epoch 24/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2429 - accuracy: 0.8972 - val_loss: 0.3450 - val_accuracy: 0.8467
Epoch 25/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2395 - accuracy: 0.8983 - val_loss: 0.3461 - val_accuracy: 0.8467
Epoch 26/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2335 - accuracy: 0.9029 - val_loss: 0.3457 - val_accuracy: 0.8388
Epoch 27/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2294 - accuracy: 0.9050 - val_loss: 0.3531 - val_accuracy: 0.8467
Epoch 28/200

In [8]:
probas_lem = modelLEM.predict(np.array(lem_test_features))
y_pred_lem = proba_to_pred(probas_lem)
results_lem = calculate_metrics(lem_test_labels, y_pred_lem)
results_lem

80/80 [==============================] - 0s 1ms/step


{'balanced_accuracy': 0.8385129352503913,
 'f1_score': 0.8451443569553806,
 'precision': 0.8125450612833454,
 'recall': 0.88046875}

# Stemmed data

In [9]:
STEMMED_DATA_PATH = os.path.join('..', 'data', 'stemmed')
stem_train_features, stem_val_features, stem_test_features, stem_train_labels, stem_val_labels, stem_test_labels, stem_vocab_size = prepare_data(
    STEMMED_DATA_PATH)

modelSTEM = buildCharCNNModel(stem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [10]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "stemmed"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelSTEM.fit(np.array(stem_train_features), np.array(stem_train_labels), batch_size=256, epochs=EPOCHS,
              validation_data=(np.array(stem_val_features), np.array(stem_val_labels)),
              callbacks=[early_stopping, save_best])

Epoch 1/200
78/81 [===========================>..] - ETA: 0s - loss: 0.5651 - accuracy: 0.7369

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 2s 19ms/step - loss: 0.5618 - accuracy: 0.7378 - val_loss: 0.4766 - val_accuracy: 0.7663
Epoch 2/200
80/81 [============================>.] - ETA: 0s - loss: 0.4349 - accuracy: 0.7946

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 1s 18ms/step - loss: 0.2849 - accuracy: 0.8722 - val_loss: 0.3495 - val_accuracy: 0.8380
Epoch 19/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2791 - accuracy: 0.8758 - val_loss: 0.3504 - val_accuracy: 0.8332
Epoch 20/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2738 - accuracy: 0.8807 - val_loss: 0.3497 - val_accuracy: 0.8410
Epoch 21/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2707 - accuracy: 0.8811 - val_loss: 0.3497 - val_accuracy: 0.8423
Epoch 22/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2606 - accuracy: 0.8886 - val_loss: 0.3523 - val_accuracy: 0.8232
Epoch 23/200
79/81 [============================>.] - ETA: 0s - loss: 0.2587 - accuracy: 0.8866

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 1s 18ms/step - loss: 0.2590 - accuracy: 0.8861 - val_loss: 0.3471 - val_accuracy: 0.8284
Epoch 24/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2504 - accuracy: 0.8938 - val_loss: 0.3513 - val_accuracy: 0.8414
Epoch 25/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2435 - accuracy: 0.8958 - val_loss: 0.3511 - val_accuracy: 0.8393
Epoch 26/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2384 - accuracy: 0.8984 - val_loss: 0.3508 - val_accuracy: 0.8380
Epoch 27/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2340 - accuracy: 0.9012 - val_loss: 0.3574 - val_accuracy: 0.8414
Epoch 28/200
81/81 [==============================] - 1s 12ms/step - loss: 0.2264 - accuracy: 0.9066 - val_loss: 0.3581 - val_accuracy: 0.8367
Epoch 29/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2242 - accuracy: 0.9068 - val_loss: 0.3623 - val_accuracy: 0.8362
Epoch 30/200

In [11]:
probas_stem = modelSTEM.predict(np.array(stem_test_features))
y_pred_stem = proba_to_pred(probas_stem)
results_stem = calculate_metrics(stem_test_labels, y_pred_stem)
results_stem

80/80 [==============================] - 0s 1ms/step


{'balanced_accuracy': 0.8350162607589984,
 'f1_score': 0.8374422187981511,
 'precision': 0.8259878419452887,
 'recall': 0.84921875}

In [12]:
# Create csv with all results

results_all = pd.DataFrame([results, results_lem, results_stem])
results_all['model'] = 'CharCNN'
results_all['dataset'] = ['simple', 'lemmatized', 'stemmed']

if not os.path.exists('results'):
    os.makedirs('results')
results_all.to_csv('results/char_cnn.csv', index=False)
results_all

,balanced_accuracy,f1_score,precision,recall,model,dataset
0,0.838130,0.843182,0.818382,0.869531,CharCNN,simple
1,0.838513,0.845144,0.812545,0.880469,CharCNN,lemmatized
2,0.835016,0.837442,0.825988,0.849219,CharCNN,stemmed
